In [1]:
# To read the PDF
from pypdf import PdfReader
#to read environment variables, list PDFs in folder etc
import os
from dotenv import load_dotenv

# to clean text from PDFs
import string

#to deal with clustering
import hdbscan
#Using langchain
from langchain import LLMChain
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain_community.embeddings import GPT4AllEmbeddings

from langchain import OpenAI

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#when playing with Vertex AI
import pandas as pd
from langchain import LLMChain
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain_google_vertexai import VertexAI

from langchain.llms import VertexAI


In [ ]:
#if getting a warning message when importing VertexAI from langchain
#!pip install -U langchain-google-vertexai

In [14]:
#!pip install langchain==0.0.342
#!pip install gpt4all==2.0.2
#!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.6 MB/s eta 0:00:0000:0100:01


In [2]:
load_dotenv()

True

In [ ]:
#specific file
pdf_file ="../raw_data/pdf_reports/2016/160602 (Marex).pdf"
reader = PdfReader(pdf_file)
page = reader.pages[0]
text =page.extract_text()
#comment = text.partition("COT")[2]
comment = text
comment = comment.lower()
comment = ''.join(char for char in comment if not char.isdigit())
for punctuation in string.punctuation:
    comment = comment.replace(punctuation, '') 
comment = comment.replace('\n'," ")
comment = comment.split("         ")[-1]
len(comment)

In [42]:
comment.split("         ")[-1]

'to say the least it was a violent week for cocoa with prices netting losses of £ respectively yet still rallying nearly points in each market off the lows charted yesterday leaving the recover y almost as equally impressive as the decline  as discussed a shoddy macro backdrop steamrolling of the  equity and commodity markets solid arrivals and perhaps an element of compulsory hedging accelerated the path lower taking prices deep into oversold territory by midweek  value based buying short coveringand a positive chart signal helped catapult prices off the lows with a weaker trending sterling providing the  additional boost for the london market  regarding short covering one other curious aspect of trading this  week involved open interest which remained curiously stable throughout the decline with any material offsets of positions only registered after the market rebounded yesterday  volumes by themselves were  another key feature with each platform averaging about  contracts a day  in

In [8]:
#listing the files from a specific folder
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("../raw_data/pdf_reports/2016") if isfile(join("../raw_data/pdf_reports/2016", f))]
onlyfiles

['161010.pdf',
 '160108.pdf',
 '160907.pdf',
 '160205.pdf',
 '160718.pdf',
 '160104.pdf',
 '161122.pdf',
 '160413.pdf',
 '161025.pdf',
 '161228.pdf',
 '161229.pdf',
 '161018.pdf',
 '161219.pdf',
 '160524.pdf',
 '160302.pdf',
 '160111.pdf',
 '160316.pdf',
 '161111.pdf',
 '160210.pdf',
 '160407.pdf',
 '161206.pdf',
 '160706.pdf',
 '160425.pdf',
 '160411.pdf',
 '160816.pdf',
 '160226.pdf',
 '160506.pdf',
 '160412.pdf',
 '160930.pdf',
 '160509.pdf',
 '160314.pdf',
 '160818.pdf',
 '160727.pdf',
 '161213.pdf',
 '161006.pdf',
 '161220.pdf',
 '160408.pdf',
 '160323.pdf',
 '160712.pdf',
 '160830.pdf',
 '161121.pdf',
 '160118.pdf',
 '160726.pdf',
 '160308.pdf',
 '161125.pdf',
 '161114.pdf',
 '160629.pdf',
 '161205.pdf',
 '160307.pdf',
 '160502.pdf',
 '160121.pdf',
 '160128.pdf',
 '160419.pdf',
 '160122.pdf',
 '160915.pdf',
 '161108.pdf',
 '161017.pdf',
 '160610.pdf',
 '160405.pdf',
 '160901.pdf',
 '160811.pdf',
 '160418.pdf',
 '160303.pdf',
 '160527.pdf',
 '160217.pdf',
 '160224.pdf',
 '160929.p

In [35]:
month = "02"
reports = [s for s in onlyfiles if s[2] + s[3] == month]
reports

['160205.pdf',
 '160210.pdf',
 '160226.pdf',
 '160217.pdf',
 '160224.pdf',
 '160209.pdf',
 '160211.pdf',
 '160218.pdf',
 '160203.pdf',
 '160219.pdf',
 '160204.pdf',
 '160225.pdf',
 '160222.pdf',
 '160202.pdf',
 '160208.pdf',
 '160216.pdf',
 '160223.pdf',
 '160201.pdf',
 '160212.pdf',
 '160229.pdf']

['160205.pdf', '160210.pdf', '160226.pdf', '160217.pdf', '160224.pdf']

In [10]:
onlyfiles[0].strip(".pdf")

'161010'

In [ ]:
#look of PDF text before cleaning
files = "160108.pdf"
text = extract_text(f"../raw_data/pdf_reports/2016/{files}", maxpages = 1)
text

In [3]:
#importaning the list from the existing CSV
comments_db = pd.read_csv("../raw_data/pdf_reports/comments_2016.csv")
comments_db = comments_db["0"].tolist()

In [10]:
%%time
#importing daily reports and cleaning text
comments_db = []
files_not_processed =[]
for files in onlyfiles:
    #text = extract_text(f"../raw_data/pdf_reports/2016/{files}", maxpages = 1)
    reader = PdfReader(f"../raw_data/pdf_reports/2016/{files}")
    page = reader.pages[0]
    text =page.extract_text()
    comment = text.partition("COT")[2]
    comment = comment.lower()
    comment = ''.join(char for char in comment if not char.isdigit())
    for punctuation in string.punctuation:
       comment = comment.replace(punctuation, '') 
    comment = comment.replace('\n'," ")
    comment = comment.split("         ")[-1]
    if len(comment) < 25:
        files_not_processed.append(files)
    else:
        comments_db.append(comment)
comments_db

UsageError: Line magic function `%%time` not found.


In [36]:
%%time
#importing daily reports but keeping text for llm
comments_db_full = []
files_not_processed =[]
for files in reports:
    #text = extract_text(f"../raw_data/pdf_reports/2016/{files}", maxpages = 1)
    reader = PdfReader(f"../raw_data/pdf_reports/2016/{files}")
    page = reader.pages[0]
    text =page.extract_text()
    comment = text.partition("COT")[2]
    comment = comment.lower()
    #comment = ''.join(char for char in comment if not char.isdigit())
    #for punctuation in string.punctuation:
    #   comment = comment.replace(punctuation, '') 
    comment = comment.replace('\n'," ")
    comment = comment.split("         ")[-1]
    if len(comment) < 25:
        files_not_processed.append(files)
    else:
        comments_db_full.append(f"date of report: {files.strip('.pdf')}; report content: {comment}")
comments_db_full

CPU times: user 2.43 s, sys: 0 ns, total: 2.43 s
Wall time: 2.43 s


['date of report: 160205; report content: email your replies to newedgecocoa@newe dge.com       new york office p:  212-278-4836 f:  212 262-3943  london office p: 207-780-6677 f:  207-676-8650       joseph p. mallaney   todd c. lunstead michael dann jonathan proctor andrew proctor michael juliano devon dinneen jamie ball graham phillipsindex + 37,635137,689net com   net ls com -delaware$413 $300(trade at settlement) 7,178 -24,9210open interest  ~    new  york  spreads    ~  friday, february 05, 2016 352,026cocoa market report open interest  3,123,399 oi nr - com + 142,417 -38,029 12,773net ls104,388 oi 308,27936,082 22,876 162,610239,603 308,2799256 9,544valid certs0.7083 1.406313620.0 4.1550$125 2.13    albany hampton roads67.78 6.571 baltimore totalkey rates dateldn volumes n.y. volumes  ~    london  spreads    ~  suprt & res. ice tas volumes net chng.05-feb-16 2,767 -7,855$263 $110 0.85 may6 new york 290,665 5-feb-16 4,280 3,425,522      while the net changes mask some of the extre

In [23]:
comments_db_full[172]

'date of report: 161117; report content: email your replies to newedgecocoa@newedge.com       new york office p:  212-278-4836 f:  212 262-3943  lo ndon office p: 207-780-6677 f:  207-676-8650       joseph p. mallaney   todd c. lunstead michael dann jonathan proctor andrew proctor  michael juliano devon dinneen jamie ball graham phillips index +  33,575 141,740  net com   net ls com - delaware $400  $330 (trade at settlement)  7,178  -21,865 0open interest  ~    new  york  spreads    ~  thursday, november 17, 2016  328,009 cocoa market report  open interest  2,595,222  oi nr - com +  -1,758 7,265  mng $ net net ls oi  284,124 18,889 163,605 284,124 11,424 valid certs 0.7458  1.4176 13375.0  4.3716 $140  2.60      albany hampton roads 67.86  6.891  baltimore total key rates  date ldn volumes n.y. volumes  ~    london  spreads    ~  suprt & res. ice tas  net chng. 11-nov-16  2,471  8,851 $280  $140  n/a mar 17  new york  260,428 17-nov-16 3,761 2,866,589       in london, prices carved ou

In [26]:
#exporting to csv
comments_db_df = pd.DataFrame(comments_db)
comments_db_df.to_csv(path_or_buf="../raw_data/pdf_reports/full_comments_2016.csv",index=False)

In [5]:
len(comments_db)

241

In [37]:
comments_full_small = comments_db_full[:5]
len(comments_full_small)

5

In [38]:
comments_full_small[3]

'date of report: 160217; report content: email your replies to newedgecocoa@newedge.com       new york office p:  212-278-4836 f:  212 262-3943  london office p: 207-780-6677 f:  207-676-8650       joseph p. mallaney   todd c. lunstead michael dann jonathan proctor andrew proctor michael juliano devon dinneen jamie ball graham phillipsindex + 35,182131,983net com   net ls com -delaware$400 $300(trade at settlement) 7,178 -23,2010open interest  ~    new  york  spreads    ~  wednesday, february 17, 2016 471,453cocoa market report open interest  3,208,011 oi nr - com + 139,412 38,569-32,953 11,804 20,954mng $ netnet ls106,459 oi 278,92531,454 21,706 155,184225,354 278,9259100 9,517valid certs0.7169 1.405513505.0 4.2125$130 2.15    albany hampton roads68.31 6.527 baltimore totalkey rates dateldn volumes n.y. volumes  ~    london  spreads    ~  suprt & res. ice tas volumes net chng.12-feb-16 2,874 -51,659$260 $110 0.80 may6 new york 322,978 17-feb-16 4,280 3,542,447      after trading on bo

In [9]:
comments_db[172]

'in london prices carved out a new annual low before late session buying in elevated volumes carried the market back  to unchanged  while technicians might throw a fanc y label on today’s prices activity  such as a “ham mer bottom”  and  point out the second consecutive “stabilizing close” the marketed failed to trade a bove wednesday’s high and still remains  within the confines of downward sloping trend chann el albeit of short duration  a strong follow thro ugh tomorrow might  validate these reversal patterns which consequentl y fall in harmony with oversold rsi’s and a normall y constructive  crossing of the stochastics  whether that amounts to anything remains to be seen as any rally in flat  price may merely  amount to a selling opportunity with system shorts comfortably positioned weekly arrivals in the ivor y coast beginning to  outpace comparative data and oodles of forward sell ing somewhere in the pipeline  regarding the world ’s top producer  rainfall data for the first ten

In [7]:
files_not_processed

['160104.pdf',
 '160203.pdf',
 '160329.pdf',
 '160204.pdf',
 '160825.pdf',
 '160331.pdf',
 '160602 (Marex).pdf']

In [121]:
comments_db_df = pd.DataFrame(comments_db)
comments_db_df.to_csv(path_or_buf="../raw_data/pdf_reports/comments_2016.csv",index=False)

In [ ]:
#embedding

In [108]:
gpt4all_embd = GPT4AllEmbeddings()

100%|█████████████████████████████████████████████████████████████████████████| 45.9M/45.9M [00:07<00:00, 6.36MiB/s]


bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [124]:
%%time
#to embed a single piece of text
query_result = gpt4all_embd.embed_query(comments_db[0])

CPU times: user 2.94 s, sys: 0 ns, total: 2.94 s
Wall time: 738 ms


In [ ]:
%%time
#to embed multiple pieces of text
doc_result = gpt4all_embd.embed_documents(comments_db)
doc_result

In [132]:
len(doc_result)

241

In [134]:
hdb = hdbscan.HDBSCAN(min_samples=3, min_cluster_size=3).fit(doc_result)
hdb.labels_

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  0,  0,  0, -1, -1, -1, -1, -1, -1, -1,
        1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1,  0, -1, -1,  0, -1,  1, -1,  0, -1, -1, -1, -1, -1,
       -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1,  1,  0, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1,
       -1, -1, -1, -1, -1

In [135]:
hdb.outlier_scores_

array([0.12377707, 0.11296396, 0.0412268 , 0.00431169, 0.1312749 ,
       0.32891157, 0.00927263, 0.3332661 , 0.05556474, 0.09642357,
       0.29447528, 0.27625234, 0.26099976, 0.16977193, 0.32913341,
       0.00865932, 0.29647474, 0.29912884, 0.25051276, 0.31677281,
       0.14363675, 0.09744723, 0.15778816, 0.04690834, 0.03407516,
       0.06260645, 0.0471761 , 0.1784305 , 0.21448001, 0.26445818,
       0.27582476, 0.03462653, 0.27315884, 0.41941459, 0.        ,
       0.02287688, 0.12291867, 0.08568888, 0.29170737, 0.30081635,
       0.11858978, 0.15316798, 0.32765181, 0.22232283, 0.18169023,
       0.        , 0.26741131, 0.06321568, 0.08327259, 0.21457622,
       0.1385791 , 0.14414101, 0.29500154, 0.0159716 , 0.03553177,
       0.15945385, 0.13402494, 0.04156014, 0.52516747, 0.16608708,
       0.39228801, 0.09859296, 0.05413311, 0.24924937, 0.22468491,
       0.        , 0.07927674, 0.08313156, 0.1350486 , 0.06015834,
       0.13316906, 0.31745659, 0.11475   , 0.07226672, 0.12145

In [ ]:
#with Vertex AI
#running into model limitations after iterating through less than 10 daily reports
llm_vertex = VertexAI(model_name="text-bison")

In [ ]:
#trying with locally hosted model

In [5]:
#Instantiating the LLM
local_path = ("../models/mistral-7b-instruct-v0.1.Q4_0.gguf")
llm = GPT4All(model=local_path)

In [6]:
#starting with a simple prompt
template = """
Please write a one sentence summary of the following text:

{comment}
"""

prompt = PromptTemplate(
    input_variables=["comment"],
    template=template
)

In [7]:
#running the summarisation as a sequence
for comment in comments_small:
    summary_prompt = prompt.format(comment=comment)
    
    num_tokens = llm.get_num_tokens(summary_prompt)
    print (f"This prompt + comment has {num_tokens} tokens")
    
    summary = llm(summary_prompt)
    
    print (f"Summary: {summary.strip()}")
    print ("\n")

/home/quentin/.pyenv/versions/3.10.6/envs/weather_checker/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This prompt + comment has 828 tokens


Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7fe536fafc70>
Traceback (most recent call last):
  File "/home/quentin/.pyenv/versions/3.10.6/envs/weather_checker/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 479, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 


Summary: The cocoa market report for Monday, October 25th indicates that system related selling and discretionary fund liquidation drove the market down. The front month's system-related sellers eventually penetrated the market by day, electing multiple sell stop orders in the process. A selling vacuum ensued as speculative liquidation added to the downward pressure on prices. However, trade-related buyers prevented the market from breaking through its recent low where many potential sell stops could be lurking for the duration of the trading session. The latest report shows that the fundamental news front did not provide any indication of an imminent selloff, with an expected low arrivals figure in the Ivory Coast. Despite this, prices fell beyond an early double bottom on the weekly charts as momentum indicators slid just north of oversold conditions. New York was surprisingly the volume leader, with highlights including Mar puts and Dec call spreads. The latter traded at an almost v

Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7fe536fafc70>
Traceback (most recent call last):
  File "/home/quentin/.pyenv/versions/3.10.6/envs/weather_checker/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 479, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 


Summary: The cocoa market experienced another day of erratic high volume trading with both extrinsic and intrinsic factors driving prices. Prices opened firm, sold off, rallied, and then skyrocketed due to realized volatility as indicated in the charts. The macro environment is blamed for the downturns but options expiration exacerbated the volatility. Commitments data from the CFTC showed a reduction in net length from the large speculative category with indexes being buyers of nearly gross contracts over the volatile period.


This prompt + comment has 534 tokens
Summary: The market in London posted a light volume inside day as prices continued to trend downward despite a modest rebound in sterling. The curve strengthened, and there was an undersupplied market on a final note. In New York, the market gapped higher but failed to gain steam due to commercial selling. Sepdec's premium eroded further with a growing net spec long position mainly concentrated in  sep. There were minor acti

In [ ]:
#now trying with OpenAI API

In [15]:
openai_api_key = os.getenv('OPENAI_API_KEY')

In [16]:
llm_openai = OpenAI(model ="gpt-3.5-turbo-instruct",temperature=0, openai_api_key=openai_api_key)

In [30]:
#using a two sentences summary and weather prompt structure
weather_prompt_template = """
You are a cocoa commodity market analyst
Please write a two sentence summary of the following text which is a daily report of the cocoa futures market
Please mention the date of the following text which is indicated at the beginning in a yy-mm-dd format.
Please mention specifically if anything about weather, precipitation, climate is indicated in the text.
{comment}
"""

prompt = PromptTemplate(
    input_variables=["comment"],
    template=weather_prompt_template
)

In [34]:
#running the summarisation as a sequence
for comment in comments_full_small:
    summary_prompt = prompt.format(comment=comment)
    
    num_tokens = llm_openai.get_num_tokens(summary_prompt)
    print (f"This prompt + comment has {num_tokens} tokens")
    
    summary = llm_openai(summary_prompt)
    
    print (f"Summary: {summary.strip()}")
    print ("\n")

This prompt + comment has 1284 tokens
Summary: The report discusses the cocoa market on October 10, 2016, including trading volumes and prices in New York and London. It also mentions a decrease in cocoa arrivals in the Ivory Coast and a decrease in the net spec long position.


This prompt + comment has 933 tokens
Summary: The report discusses the recent volatility in the cocoa market, with prices dropping and then rebounding due to various factors such as a weak macro backdrop, solid arrivals, and short covering. Open interest and trading volumes were also high, with options trading matching the entire annual global output. Implied volatilities were up and there was a less than expected reduction in speculative length.


This prompt + comment has 1168 tokens
Summary: The report discusses the current state of the cocoa market, including trading volume and price action. It also mentions a potential surplus and weather conditions in Cote d'Ivoire.


This prompt + comment has 871 tokens


In [39]:
comments_full_small[4]

'date of report: 160718; report content: email your replies to newedgecocoa@newedge.com       new york office p:  212-278-4836 f:  212 262-3943  london office p: 207-780-6677 f:  207-676-8650       joseph p. mallaney   todd c. lunstead michael dann jonathan proctor andrew proctor michael juliano devon dinneen jamie ball graham phillipsindex + 33,30039,827 115,126117,503net com   net ls com -delaware$500 $425(trade at settlement) 7,178 -48,9080open interest  ~    new  york  spreads    ~  monday, july 18, 2016 162,024cocoa market report open interest  3,236,766 oi nr - com + 150,129 144,841 65,286-55,800 -53,738 -44,603 159,729 32,706 29,258 46,361mng $ netnet ls94,329 91,103 249,524oi 260,347 32,42250,056 43,160 166,41148,930 249,523213,944 205,072 260,3466927 7,132 6,4986270valid certs0.7593 1.348213085.0 3.9750$145 2.45     alban y hampton roads67.15 6.701 baltimore totalkey rates dateldn volumes n.y. volumes  ~    london  spreads    ~  suprt & res. ice tas net chng.15-jul-16 3,050 -2

In [17]:
#using a more complex prompt for the summarize chain using map_reduce with OpenAI
map_prompt_template = """
                      Write a summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Write a concise summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

In [18]:
# create LLM chain
map_reduce_chain = load_summarize_chain(
    llm_openai,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=False,
    verbose=True
)

In [39]:
%%time
#splitting documents
#split in chunks of 1000 characters 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=100)
docs = text_splitter.create_documents(comments_full_small)
docs = text_splitter.split_documents(docs)

CPU times: user 12.8 ms, sys: 754 µs, total: 13.5 ms
Wall time: 13.2 ms


In [40]:
num_docs = len(docs)
for doc in range(num_docs):
    num_tokens = llm_openai.get_num_tokens(docs[doc].page_content)
    print (f"Now we have {num_docs} documents and the document{doc} has {num_tokens} tokens")

Now we have 10 documents and the document0 has 689 tokens
Now we have 10 documents and the document1 has 172 tokens
Now we have 10 documents and the document2 has 676 tokens
Now we have 10 documents and the document3 has 311 tokens
Now we have 10 documents and the document4 has 716 tokens
Now we have 10 documents and the document5 has 137 tokens
Now we have 10 documents and the document6 has 686 tokens
Now we have 10 documents and the document7 has 342 tokens
Now we have 10 documents and the document8 has 710 tokens
Now we have 10 documents and the document9 has 252 tokens


In [34]:
token=llm_openai.get_num_tokens(docs[0].page_content)
token

3

In [26]:
%%time
map_reduce_outputs = map_reduce_chain({"input_documents": docs})



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

                      Write a summary of this chunk of text that includes the main points and any important details.
                      trade at settlement     open interest      new  york  spreads      monday october     cocoa market report  open interest    oi nr  com     index      net com   net ls com  delaware       in london a heavy bout of system related sell ing and discretionary fund liquidation were the mai n culprits in driving the ma rket  down  basis the front month  system related selling appeared from the onset eventually  penetrating  basis mar the   day moving average electing multiple sell stop s in the process a selling vacuum ensued as spec l iquidation added a furt her leg  lower with market finally finding support at  b asis mar  trade related buyers prevented the market from brea king through   basis mar the recent low where many potent ial sell stops

In [27]:
map_reduce_outputs

{'input_documents': [Document(page_content='trade at settlement     open interest      new  york  spreads      monday october     cocoa market report  open interest    oi nr  com     index      net com   net ls com  delaware       in london a heavy bout of system related sell ing and discretionary fund liquidation were the mai n culprits in driving the ma rket  down  basis the front month  system related selling appeared from the onset eventually  penetrating  basis mar the   day moving average electing multiple sell stop s in the process a selling vacuum ensued as spec l iquidation added a furt her leg  lower with market finally finding support at  b asis mar  trade related buyers prevented the market from brea king through   basis mar the recent low where many potent ial sell stops could be lurking for the duration of  the trading session n earby  structure finally showed signs of weakness during the flat price sell off as its prem ium quickly deteriorated with decmar  closing at  pr

In [29]:
map_reduce_outputs.keys()

dict_keys(['input_documents', 'output_text'])

In [31]:
print(map_reduce_outputs["output_text"])


- Cocoa market report from Monday, October
- Factors driving market down: open interest, spreads, system-related selling, discretionary fund liquidation
- Market found support at certain level, but potential sell stops lurking
- Fundamental news did not indicate sell-off, net spec long position diminished
- New York prices suffered greater losses, front month spread plunged and surrendered a third of its premium
- Inverted structure and short-term physical tightness inconsistent with recent downward movements in flat price
- Erosion on decmar compounded overall market weakness and emboldened systematic community
- Today's price activity was utter devastation from technical perspective
- Extremely negative price activity, falling below recent lows and dropping beyond double bottom on weekly charts
- Momentum indicators suggest potential relief rally, but market focused on potential surplus
- High volume in options market, notable trades including March puts and December call spreads
- 

In [11]:
#trying map_reduce with local llm
# create LLM chain
map_reduce_chain_local = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=False,
    verbose=True
)
num_docs = len(docs)
num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)
print (f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")

/home/quentin/.pyenv/versions/3.10.6/envs/weather_checker/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now we have 8 documents and the first one has 584 tokens


In [ ]:
%%time
map_reduce_outputs_local = map_reduce_chain_local({"input_documents": docs})



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

                      Write a summary of this chunk of text that includes the main points and any important details.
                      trade at settlement     open interest      new  york  spreads      monday october     cocoa market report  open interest    oi nr  com     index      net com   net ls com  delaware       in london a heavy bout of system related sell ing and discretionary fund liquidation were the mai n culprits in driving the ma rket  down  basis the front month  system related selling appeared from the onset eventually  penetrating  basis mar the   day moving average electing multiple sell stop s in the process a selling vacuum ensued as spec l iquidation added a furt her leg  lower with market finally finding support at  b asis mar  trade related buyers prevented the market from brea king through   basis mar the recent low where many potent ial sell stops

Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7f9f37eeb760>
Traceback (most recent call last):
  File "/home/quentin/.pyenv/versions/3.10.6/envs/weather_checker/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 479, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 
Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7f9f37eeb760>
Traceback (most recent call last):
  File "/home/quentin/.pyenv/versions/3.10.6/envs/weather_checker/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 479, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 
Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7f9f37eeb760>
Traceback (most recent call last):
  File "/home/quentin/.pyenv/versions/3.10.6/envs/weather_checker/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 479, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 


In [ ]:
print(map_reduce_outputs_local["output_text"])

In [ ]:
#now using HuggingFace summarisation model

In [43]:
from transformers import pipeline
pipe_summarise = pipeline("summarization", model = "sshleifer/distilbart-xsum-12-6")
article = pipe_summarise(comments_full_small[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (1258 > 1024). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self

In [137]:
%%time
from gensim.models import Word2Vec
word2vec = Word2Vec(sentences=comments_db)
wv = word2vec.wv

CPU times: user 118 ms, sys: 27.1 ms, total: 145 ms
Wall time: 115 ms


In [138]:
wv.key_to_index

{' ': 0,
 'e': 1,
 't': 2,
 'n': 3,
 'o': 4,
 'a': 5,
 'i': 6,
 'r': 7,
 's': 8,
 'l': 9,
 'd': 10,
 'h': 11,
 'c': 12,
 'p': 13,
 'm': 14,
 'u': 15,
 'g': 16,
 'f': 17,
 'y': 18,
 'w': 19,
 'v': 20,
 'b': 21,
 'k': 22,
 'x': 23,
 'j': 24,
 'q': 25,
 '’': 26,
 '£': 27,
 '\x0c': 28,
 'z': 29,
 '–': 30}

In [ ]:
#below are trials to extract text from the PDFs

In [66]:
text = extract_text('raw_data/pdf_reports/210113.pdf', maxpages = 1)
text

'COCOA MARKET REPORT\n\n   Arb Settle Basis Ldn Close on 20-12-24\n\nWednesday, January 13, 2021\n\nSource:  Bloomberg and the I.C.E.\n\nI.C.E EU\nCLOSE\n1703\n1648\n1631\n1619\n1626\n1623\n1625\n1627\n1629\n1636\n\nMAR 21\nMAY 21\nJUL 21\nSEP 21\nDEC 21\nMAR 22\nMAY 22\nJUL 22\nSEP 22\nDEC 22\nLondon Due: \n\n9\n7\n3\n0\n1\n1\n0\n-1\n-1\n-1\nMAY 21\n\nHI\n1,721\n1,663\n1,644\n1,632\n1,637\n1,634\n1,635\n1,636\n\nLOW\n1,683\n1,633\n1,620\n1,614\n1,619\n1,618\n1,624\n1,630\n\n-£8\n\nLower\n\nVOLUME\n5252\n5058\n3610\n1340\n862\n170\n90\n50\n\nOPEN INTEREST \n248\n-734\n92\n376\n-99\n67\n-9\n22\n\n73,478\n39,523\n29,528\n27,699\n34,104\n10,766\n4,240\n2,074\n406\n18\n221,836\n\n4\n-33\n\n16,432\nStock Market Indices\n31060.47\nDJIA\n3809.84\nS&P500\nNASDAQ\n13128.95\nFTSE 100 IDX  6745.52\nNIKKEI225\n28456.6\nCommodity Indices:\n471.72\nCCI\n\nBCOM\n\nGSCI\n\nTRJ CRB\n\nBltc Frt Idx\n\n80.61\n\n432.40\n\n175.19\n\n1849\n\n90.343\n\nDXY\nCommodities (2nd Pos.):\nSugar\n\n14.95\n\nLDN VOLU

In [67]:
len(text)

9739

In [99]:
comment = text.partition("COT")[2]
comment

'\nw/Opt.\n\nEU COT\nw/Opt.\n\nValid\n2,000\n\n5-Jan\n\nExp\n3,290\n\nCom +\n69,872\n\nCom +\n5-Jan 155,804\n\nSDU (10 mts)\nNon-Tend\n254\n\nLive\n\nCom -\n135,389\n\nCom -\n176,155\n\nCom Net\n-65,517\n\nCom Net\n-20,351\n\nTotal\n5,708\n\nSwp +\n22,873\n\nSwp +\n17,953\n\nValid\n20\n\nSwp -\n4,646\n\nSwp -\n17,527\n\nExp\n73\n\nSwp Net\n18,227\n\nSwp Net\n426\n\nLDU (100 mts)\nNon-Tend\n6\n\nMng $ +\n50,955\n\nMng $ +\n19,741\n\nLive\n\nMng $ -\n23,808\n\nMng $ -\n3,074\n\nTotal\n99\n\nValid\n32\nMng $ Net\n27,147\n\nMng $ Net\n16,667\n\nExp\n32\n\nOthr +\n18,893\n\nOthr +\n3,360\n\nBDU (1,000 mts)\nNon-Tend\n\nOthr -\n5,963\n\nOthr -\n884\n\nLive\n1\n\nOthr Net\n12,930\n\nOthr Net\n2,476\n\nTotal\n65\n\nNR +\n14,778\n\nNR +\n3,680\n\nValid\n54,000\n\nNR -\n7,564\n\nNR -\n2,898\n\nExp\n72,200\n\nNR Net\n7,214\n\nNR Net\n782\n\nTotal\nNon-Tend\n3,140\n\nOI\n254,195\n\nOI\n253,288\n\nLive\n1,000\n\nNet Spec\n65,518\n\nNet Spec\n20,351\n\nALL\n130,340\n\nChange\n1,854\n\nChange\n-5,765

In [104]:
comment = comment.lower()
comment = ''.join(char for char in comment if not char.isdigit())

for punctuation in string.punctuation:
   comment = comment.replace(punctuation, '') 

comment = comment.replace('\n',"")
len(comment)

4076

In [105]:
comment

'wopteu cotwoptvalidjanexpcom com jan sdu  mtsnontendlivecom com com netcom nettotalswp swp validswp swp expswp netswp netldu  mtsnontendmng  mng  livemng  mng  totalvalidmng  netmng  netexpothr othr bdu  mtsnontendothr othr liveothr netothr nettotalnr nr validnr nr expnr netnr nettotalnontendoioilivenet specnet specallchangechange     although a rare bird industry selling is a feature which occasionally flutters into view even on cocoa’s wilder moors – but bestto have the high strength binoculars at the ready though as it does not tend to indulge its hides of twitchers for too long even rarer –in fact ornithologically speaking a brandnew species – the market was treated to the sight of origin buying today as a bloomberg story winged its way across the newswires reporting the possibility of an ivorian government purchase of  tons not ‘tonnes’ according to the text from farmers and cooperatives who are still long it is true that when the wind blows even a turkey can fly and maybe  they 

In [102]:
from transformers import pipeline
summarizer = pipeline("summarization", model="Falconsai/text_summarization")

config.json: 100%|██████████████████████████████████████████████████████████| 1.49k/1.49k [00:00<00:00, 2.73MB/s]
model.safetensors: 100%|██████████████████████████████████████████████████████| 242M/242M [00:24<00:00, 9.89MB/s]
generation_config.json: 100%|████████████████████████████████████████████████████| 112/112 [00:00<00:00, 306kB/s]
tokenizer_config.json: 100%|████████████████████████████████████████████████| 2.32k/2.32k [00:00<00:00, 5.84MB/s]
spiece.model: 100%|███████████████████████████████████████████████████████████| 792k/792k [00:00<00:00, 8.18MB/s]
tokenizer.json: 100%|███████████████████████████████████████████████████████| 2.42M/2.42M [00:00<00:00, 3.42MB/s]
special_tokens_map.json: 100%|██████████████████████████████████████████████| 2.20k/2.20k [00:00<00:00, 9.01MB/s]


In [103]:
print(summarizer(comment, max_length=230, min_length=30, do_sample=False))

Token indices sequence length is longer than the specified maximum sequence length for this model (996 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'the market was treated to the sight of origin buying today as a bloomberg story winged its way across the newswires reporting the possibility of an ivorian government purchase of tons not ‘tonnes’ according to the text from farmers and cooperatives who are still long . there could be a few to those ‘familiar with the matter’ might be their intentions for the other tons allegedly dotted around the place or where that falls within the plans to ‘roll’ more than tons of unsold main crop into the'}]


In [123]:
output_string = StringIO()
with open('raw_data/pdf_reports/210113.pdf', 'rb') as fin:
    extract_text_to_fp(fin, output_string, laparams=LAParams(),
                       output_type='text', codec=None, maxpages=1)
output_string.getvalue()

FileNotFoundError: [Errno 2] No such file or directory: 'raw_data/pdf_reports/210113.pdf'

In [15]:
# Create a function to extract text
def text_extraction(element):
    # Extracting the text from the in-line text element
    line_text = element.get_text()
    
    # Find the formats of the text
    # Initialize the list with all the formats that appeared in the line of text
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Iterating through each character in the line of text
            for character in text_line:
                if isinstance(character, LTChar):
                    # Append the font name of the character
                    line_formats.append(character.fontname)
                    # Append the font size of the character
                    line_formats.append(character.size)
    # Find the unique font sizes and names in the line
    format_per_line = list(set(line_formats))
    
    # Return a tuple with the text in each line along with its format
    return (line_text, format_per_line)

In [22]:
# Extracting tables from the page

def extract_table(pdf_path, page_num, table_num):
    # Open the pdf file
    pdf = pdfplumber.open(pdf_path)
    # Find the examined page
    table_page = pdf.pages[page_num]
    # Extract the appropriate table
    table = table_page.extract_tables()[table_num]
    return table

# Convert table into the appropriate format
def table_converter(table):
    table_string = ''
    # Iterate through each row of the table
    for row_num in range(len(table)):
        row = table[row_num]
        # Remove the line breaker from the wrapped texts
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        # Convert the table into a string 
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    # Removing the last line break
    table_string = table_string[:-1]
    return table_string

In [23]:
# Find the PDF path
pdf_path = 'raw_data/pdf_reports/210113.pdf'
# create a PDF file object
pdfFileObj = open(pdf_path, 'rb')
# create a PDF reader object
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

# Create the dictionary to extract text from each image
text_per_page = {}

# We extract the pages from the PDF
for pagenum, page in enumerate(extract_pages(pdf_path)):
    #Initialize the variables needed for the text extraction from the page
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    line_format = []  
    text_from_tables = []
    page_content = []
    # Initialize the number of the examined tables
    table_num = 0
    first_element= True
    table_extraction_flag= False
    #Open the pdf file
    pdf = pdfplumber.open(pdf_path)
    #Find the examined page
    page_tables = pdf.pages[pagenum]
    # Find the number of tables on the page
    tables = page_tables.find_tables()
    
    # Find all the elements
    page_elements = [(element.y1, element) for element in page._objs]
    # Sort all the elements as they appear in the page 
    page_elements.sort(key=lambda a: a[0], reverse=True)
    # Find the elements that composed a page
    for i,component in enumerate(page_elements):
        # Extract the position of the top side of the element in the PDF
        pos= component[0]
        # Extract the element of the page layout
        element = component[1]
        
        # Check if the element is a text element
        if isinstance(element, LTTextContainer):
            # Check if the text appeared in a table
            if table_extraction_flag == False:
                # Use the function to extract the text and format for each text element
                (line_text, format_per_line) = text_extraction(element)
                # Append the text of each line to the page text
                page_text.append(line_text)
                # Append the format for each line containing text
                line_format.append(format_per_line)
                page_content.append(line_text)
            else:
                # Omit the text that appeared in a table
                pass

           # Check the elements for tables
        if isinstance(element, LTRect):
            # If the first rectangular element
            if first_element == True and (table_num+1) <= len(tables):
                # Find the bounding box of the table
                lower_side = page.bbox[3] - tables[table_num].bbox[3]
                upper_side = element.y1 
                # Extract the information from the table
                table = extract_table(pdf_path, pagenum, table_num)
                # Convert the table information in structured string format
                table_string = table_converter(table)
                # Append the table string into a list
                text_from_tables.append(table_string)
                page_content.append(table_string)
                # Set the flag as True to avoid the content again
                table_extraction_flag = True
                # Make it another element
                first_element = False
                # Add a placeholder in the text and format lists
                page_text.append('table')
                line_format.append('table')

In [39]:
tables

In [46]:
page_elements[1][1]

<LTRect 16.080,685.920,596.640,687.120>